In [1]:
import numpy as np
import random

In [2]:
class SnakeNode():
    def __init__(self, parent, x, y):
        self.parent = parent
        self.x = x
        self.y = y
    def SetX(self, newx):
        self.x = newx
    def SetY(self, newy):
        self.y = newy
    def SetParent(self, newparent):
        self.parent = newparent
    def Pos(self):
        return (self.x, self.y)
    def Index(self):
        return (self.y, self.x)

In [3]:
class Snake():

    def __init__(self, HeadX, HeadY):
        self.head = SnakeNode(None, HeadX, HeadY)
        self.tail = self.head

    def MoveBodyForward(self):
        currentnode = self.tail
        while (currentnode.parent != None):
            parentpos = currentnode.parent.Pos()
            currentnode.SetX(parentpos[0])
            currentnode.SetY(parentpos[1])
            currentnode = currentnode.parent

    def Move(self, direction):
        (oldtailx, oldtaily) = self.tail.Pos()
        self.MoveBodyForward()
        headpos = self.head.Pos()
        if direction == 0:
            self.head.SetY(headpos[1] - 1)
        elif direction == 1:
            self.head.SetX(headpos[0] + 1)
        elif direction == 2:
            self.head.SetY(headpos[1] + 1)
        elif direction == 3:
            self.head.SetX(headpos[0] - 1)
        
        return (oldtailx, oldtaily, *self.head.Pos())
    
    def NewHead(self, headx, heady):
        newhead = SnakeNode(None, headx, heady)
        self.head.SetParent(newhead)
        self.head = newhead

    def Head(self):
        return self.head
    def Tail(self):
        return self.tail

In [4]:
#from typing import Collection
class Game():
    x = 1
    def __init__(self):
        self.foodval = 10
        self.headval = 5
        self.bodyval = 2
        self.obstacleval = 1
        self.length = 1
        self.board = np.zeros((20,20), dtype=int)
        self.SpawnObstacle()
        self.SpawnSnake()
        self.SpawnFood()
        self.Extractfeatures()




    def SpawnObstacle(self):
        obstaclespawns = []
        obstaclespawns.append(np.random.randint(1, 7, size=(2)))
        obstaclespawns.append(np.random.randint(7, 14, size=(2)))
        obstaclespawns.append(np.random.randint(14, 19, size=(2)))
        for i in obstaclespawns:
            b0 = i
            b1 = (i[0] + 1, i[1])
            b2 = (i[0], i[1] + 1)
            b3 = (i[0] + 1, i[1] + 1)
            self.board[b0[0], b0[1]] = self.obstacleval
            self.board[b1[0], b1[1]] = self.obstacleval
            self.board[b2[0], b2[1]] = self.obstacleval
            self.board[b3[0], b3[1]] = self.obstacleval



    def SpawnSnake(self):
        snakespawns = []
        for index, cell in np.ndenumerate(self.board):
            if cell == 0:
                snakespawns.append(index)
        self.snakespawn = random.choice(snakespawns)
        startx = self.snakespawn[0]
        starty = self.snakespawn[1]
        #self.board[startx, starty] = self.headval
        self.snake = Snake(startx, starty)
        return startx, starty


    def SpawnFood(self):
        foodspawns = []
        for index, cell in np.ndenumerate(self.board):
            if cell == 0:
                foodspawns.append(index)
        self.foodspawn = random.choice(foodspawns)
        self.board[self.foodspawn] = self.foodval

    def DirectionHead(self, direction):
        (headx, heady) = self.snake.Head().Pos()
        if direction == 0:
            heady -= 1
        elif direction == 1:
            headx += 1
        elif direction == 2:
            heady += 1
        elif direction == 3:
            headx -= 1
        return (headx, heady)
    
    def CollisionCheck(self, direction):
        headx, heady = self.DirectionHead(direction)

        if (headx == -1) or (headx == 20) or (heady == -1) or (heady == 20):
            return True
        elif self.board[heady, headx] == self.bodyval:
            return True
        elif self.board[heady, headx] == self.obstacleval:
            return True
        return False

    def Extractfeatures(self):
        self.features = np.zeros(8, dtype= int)
        for i in range(4):
            self.features[i] = self.CollisionCheck(i)
        self.features[4:] = self.FoodDirection()
    
    def FoodDirection(self):
        foodvector = np.array(self.foodspawn) - np.array(self.snake.Head().Index())
        foodfeatures = np.zeros(4, dtype= int)

        if foodvector[0] < 0:
            foodfeatures[0] = 1
        elif foodvector[0] > 0:
            foodfeatures[2] = 1
        if foodvector[1] > 1:
            foodfeatures[1] = 1
        elif foodvector[1] < 1:
            foodfeatures[3] = 1
        
        return foodfeatures

    def Move(self, direction):
        gameisover = False
        if not self.CollisionCheck(direction):
            if self.FoodDirection()[direction] == 1:
                reward = 1
            else:
                reward = 0
            (headx, heady) = self.snake.Head().Pos()
            self.board[heady, headx] = self.bodyval


            potx, poty = self.DirectionHead(direction)

            if self.board[poty, potx] == self.foodval:
                self.snake.NewHead(potx, poty)
                self.board[poty, potx] = self.headval
                self.SpawnFood()
                self.length += 1 
                reward = 2
            else:
                (oldtailx, oldtaily, newheadx, newheady) = self.snake.Move(direction)
                self.board[oldtaily, oldtailx] = 0
                self.board[newheady, newheadx] = self.headval
        else:
            reward = -2
            gameisover = True
        
        self.Extractfeatures()
        return gameisover, self.length


    def display(self):
        for i in range(22):
            print('-', end='')

        for i in range(20):
            print('\n|', end='')
            for j in range(20):
                if self.board[i, j] == 0:
                    print(' ', end='')
                elif self.board[i, j] == self.headval:
                    print('S', end='')
                elif self.board[i, j] == self.bodyval:
                    print('#', end='')
                elif self.board[i, j] == self.foodval:
                    print('O', end='')
                elif self.board[i, j] == self.obstacleval:
                    print('x', end='')
            print('|', end='')
        print()
        for i in range(22):
            print('-', end='')
        print()

In [5]:

if __name__ == "__main__":
    game = Game()
    game.display()
    while True:
        direction = input("Input Direction (w,a,s,d or q to quit): ")
        if direction == 'w':
            gameOver, score = game.Move(0)
        elif direction == 'a':
            gameOver, score = game.Move(3)
        elif direction == 's':
            gameOver, score = game.Move(2)
        elif direction == 'd':
            gameOver, score = game.Move(1)
        elif direction == 'q':
            break
        if gameOver:
            print("Game Over, Score:", score)
            break
        else:
            game.display()

----------------------
|                    |
|     O              |
|                    |
|                    |
| xx                 |
| xx                 |
|                    |
|                    |
|                    |
|       xx           |
|       xx           |
|                    |
|                    |
|                    |
|                  xx|
|                  xx|
|                    |
|                    |
|                    |
|                    |
----------------------
Input Direction (w,a,s,d or q to quit): w
----------------------
|                    |
|     O              |
|                    |
|                    |
| xx                 |
| xx                 |
|       S            |
|                    |
|                    |
|       xx           |
|       xx           |
|                    |
|                    |
|                    |
|                  xx|
|                  xx|
|                    |
|                    |
|              

KeyboardInterrupt: ignored